In [ ]:
!pip install glove-python-binary

In [ ]:
# !pip install tokenizers

# Imports

In [ ]:
import numpy as np
import gc
import pandas as pd
import nltk
import re
from joblib import Parallel, delayed
from gensim.models import FastText,Word2Vec
from glove import Glove,Corpus
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from scipy import sparse
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,PreTrainedTokenizerFast,pipeline)
from datasets import Dataset
from sklearn.linear_model import Ridge
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

# Utils

In [ ]:
def tokens2sentence(x):
    return ' '.join(x)

def dummy_fun(doc):
    return doc

def get_words_w2v_vec(model,sent):#使用word2vec获取整句话的vec
    return np.array(list(map(lambda x:model.wv[x],sent)),dtype=float).mean(axis=0)

def get_words_glove_vec(model,sent):#使用glove获取整句话的vec
    return np.array(list(map(lambda x:model.word_vectors[model.dictionary[x]],sent)),dtype=float).mean(axis=0)


In [ ]:
import numpy as np
import gc
import pandas as pd
import re
import torch
from joblib import Parallel, delayed
from gensim.models import FastText,Word2Vec
from glove import Glove,Corpus
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,PreTrainedTokenizerFast,pipeline)
from torch.utils.data import DataLoader, Dataset
from sklearn.linear_model import Ridge
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

def tokens2sentence(x):
    return ' '.join(x)

def dummy_fun(doc):
    return doc

def get_words_w2v_vec(model,sent):#使用word2vec获取整句话的vec
    return np.array(list(map(lambda x:model.wv[x],sent)),dtype=float).mean(axis=0)

def get_words_glove_vec(model,sent):#使用glove获取整句话的vec
    return np.array(list(map(lambda x:model.word_vectors[model.dictionary[x]],sent)),dtype=float).mean(axis=0)

def prepare_input(tokenizer,text):
    inputs = tokenizer(text, 
                       max_length=128, 
                        return_tensors='pt',
                        padding='max_length',
                        truncation=True)
    return inputs
class TrainDataset(Dataset):
    def __init__(self, tokenizer, texts):
        self.tokenizer = tokenizer
        self.texts = texts.values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.tokenizer,  
                               self.texts[item])
        return inputs
    
def model_infer(model,data,device):
    preds = []
    model.eval()
    model.to(device)
    for inputs in data:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(**inputs)
        preds.append(y_preds['hidden_states'][0].detach().to('cpu').numpy().mean(1))
    predictions = np.concatenate(preds,axis=0)
    return predictions

class NLP_feature():
    def __init__(self):
        self.tokenizers = {}
        self.embedded_texts = {}
        self.model_name = 'prajjwal1/bert-tiny'
        self.zero_shot_model = 'typeform/mobilebert-uncased-mnli' #joeddav/xlm-roberta-large-xnli
        self.embeddings = {}
        self.corpus_model = {}
        self.encoders   = {}
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.candidate_labels = {}
        self.pipline = None
        self.n_clusters = 2
        self.do_mlm = False
        self.mlm_epochs = 2
        self.emb_size   = 32
        self.use_tokenizer = False
        self.text_columns_def = None
        self.y = None
        self.task = None
        self.embedding_mode = None
    def fit(self,df,text_columns_def,use_tokenizer,embedding_mode,task,y=None,candidate_labels=None):
        self.task = task
        self.use_tokenizer=use_tokenizer
        self.text_columns_def=text_columns_def
        self.embedding_mode=embedding_mode
        self.candidate_labels=candidate_labels
        self.y = y
        df = df.loc[:,text_columns_def]
        for col in text_columns_def:
            df[col] = df[col].apply(str)
        if self.task == 'zero-shot-classification':
            self.pipeline = pipeline(self.task,model=self.zero_shot_model )
            return
        ## 训练分词器，如果不使用，则默认使用空格分词
        if self.use_tokenizer:
            self.fit_tokenizers(df)
        if self.embedding_mode != 'Bert':
            for column in self.text_columns_def:
                df[f'{column}_tokenized_ids'] = self.tokenize(df,column)
#         return df
        ## 训练embedding,初步确定五种： TFIDF、FastText、Word2Vec、Glove、 BertEmbedding,训练的embedding model数量与文本特征列数量相同,使用字典存储，索引为特征列名
        self.fit_embeddings(df)
        ## 根据task训练特征提取器,训练的encoder model数量与文本特征列数量相同,使用字典存储，索引为特征列名, 与每一列的embedding model也是一一对应
        ## 目前支持：有监督回归数值特征、无监督KNN距离特征、无监督关键词离散特征(语义，情感，。。。）
        ## 特殊任务：根据两两对比数据 生成 每个数据的具体得分，比如通过成对评论的恶意(替换成任何一种语义程度都可以)比较，生成单个评论的恶意程度
        ## 注意： 关键词离散特征和特殊任务目前只支持使用深度模型，无法自选tokenizer和embedding
        return self.fit_encoders(df,y)
        ## 使用提取器处理文本数据生成新特征
    
    def fit_tokenizers(self,df):
        raw_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
        raw_tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False)
        raw_tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
        special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
        trainer = trainers.WordPieceTrainer(vocab_size=500, special_tokens=special_tokens)
#         df = pd.concat([df_train_new[['comment_text']]])

        def micro_tokenizer(text_df,name):
            text = pd.concat([text_df])
            dataset = Dataset.from_pandas(text)
            def get_training_corpus():
                for i in range(0, len(dataset), 1000):
                    yield dataset[i : i + 1000][name]
            raw_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)
            tokenizer = PreTrainedTokenizerFast(
                tokenizer_object=raw_tokenizer,
                unk_token="[UNK]",
                pad_token="[PAD]",
                cls_token="[CLS]",
                sep_token="[SEP]",
                mask_token="[MASK]",
            )
            return tokenizer
        
        for column in self.text_columns_def:
            print(f'Fitting column: {column} tokenizer')
            if self.embedding_mode != 'Bert': 
                self.tokenizers.update({column:micro_tokenizer(df[[column]],column)})
            else:
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                self.tokenizers.update({column:tokenizer})
            
    def tokenize(self,df,column):
        if self.use_tokenizer:
#             for column in self.text_columns_def:
            print(f'Tokenizing column: {column}')
            tokenizer = self.tokenizers[column]
            return list(map(lambda x: self.tokenizers[column].convert_ids_to_tokens(x),
                            self.tokenizers[column](df[column].to_list())['input_ids']))
        else:
            return [text.split(' ') for text in df[column].to_list()]
    
    def fit_embeddings(self,df):
        if self.embedding_mode == 'TFIDF':
            def micro_tfidf(text_df,name):
                vectorizer = TfidfVectorizer(
                    analyzer = 'word',
                    tokenizer = dummy_fun,
                    preprocessor = dummy_fun,
                    token_pattern = None
                )
                self.embeddings.update({name:vectorizer.fit(text_df.to_list())})
            for column in self.text_columns_def:
                print(f'Fitting column: {column} tfidf embedding')
                micro_tfidf(df[f'{column}_tokenized_ids'],column)
                
        elif self.embedding_mode == 'FastText':
            def micro_fasttext(text_df,name):
                model = FastText(vector_size=self.emb_size)
#                 text_df = text_df.apply(tokens2sentence)
                model.build_vocab(text_df)
                model.train(
                    text_df, epochs=model.epochs,
                    total_examples=model.corpus_count, total_words=model.corpus_total_words,
                )
                self.embeddings.update({name:model})
            for column in self.text_columns_def:
                print(f'Fitting column: {column} fasttext embedding')
                micro_fasttext(df[f'{column}_tokenized_ids'],column)
        elif self.embedding_mode == 'Word2Vec':
            def micro_word2vec(text_df,name):
                model = Word2Vec(vector_size=self.emb_size,min_count=-1)
                model.build_vocab(text_df)
#                 model.wv.key_to_index = self.tokenizers[name].vocab
                model.train(
                    text_df, epochs=model.epochs,
                    total_examples=model.corpus_count, total_words=model.corpus_total_words,
                )
                self.embeddings.update({name:model})
            for column in self.text_columns_def:
                print(f'Fitting column: {column} word2vec embedding')
                micro_word2vec(df[f'{column}_tokenized_ids'],column)
        elif self.embedding_mode == 'Glove':
            def micro_glove(text_df,name):
                corpus_model = Corpus()
                corpus_model.fit(text_df,window=10,ignore_missing=False)
                model = Glove(no_components=self.emb_size)
                model.fit(corpus_model.matrix,epochs=10)
                model.add_dictionary(corpus_model.dictionary)
                self.embeddings.update({name:model})
                self.corpus_model.update({name:corpus_model})
            for column in self.text_columns_def:
                print(f'Fitting column: {column} glove embedding')
                micro_glove(df[f'{column}_tokenized_ids'],column)
        elif self.embedding_mode == 'Bert':
            def micro_bert(text_df,name):
                text_df = text_df.apply(lambda x: x.replace('\n',''))
                text  = '\n'.join(text_df.tolist())
                with open('text.txt','w') as f:
                    f.write(text)
                model = AutoModelForMaskedLM.from_pretrained(self.model_name)
                model.to(self.device)
                model_path = f'./{name}_transformer'
                train_dataset = LineByLineTextDataset(
                    tokenizer=self.tokenizers[name],
                    file_path="text.txt",
                    block_size=128)
                data_collator = DataCollatorForLanguageModeling(
                    tokenizer=self.tokenizers[name], mlm=True, mlm_probability=0.15)
                training_args = TrainingArguments(
                    output_dir=model_path,
                    num_train_epochs=self.mlm_epochs,
                    per_device_train_batch_size=8,
                    per_device_eval_batch_size=8,
                    evaluation_strategy= 'no',
                    save_strategy='no',
                    report_to = "none")
                trainer = Trainer(
                    model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=train_dataset,
                    eval_dataset=None)
                if self.do_mlm:
                    trainer.train()
                trainer.save_model(model_path)
                self.embeddings.update({name:model_path})
            for column in self.text_columns_def:
                print(f'Fitting column: {column} bert embedding')
                micro_bert(df[column],column)
        else:
            raise NotImplementedError
    def emb_text(self,raw_df,name):
#             tokenizer = self.tokenizers[name]
        raw_df[name] = raw_df[name].apply(str)
        embedding = self.embeddings[name]
        embedded_text = None
        if self.embedding_mode == 'TFIDF':
            embedded_text = embedding.transform(self.tokenize(raw_df,name))
        elif self.embedding_mode == 'FastText':
            embedded_text = sparse.csr_matrix(embedding.wv[
                list(map(lambda x:tokens2sentence(x),self.tokenize(raw_df,name)))
            ])
        elif self.embedding_mode == 'Word2Vec':
            text_df = self.tokenize(raw_df,name)
            embedding.build_vocab(text_df)
            embedding.train(
                    text_df, epochs=embedding.epochs,
                    total_examples=embedding.corpus_count, total_words=embedding.corpus_total_words,
                )
            embedded_text = sparse.csr_matrix(np.array(
                list(map(lambda x:get_words_w2v_vec(embedding,x),text_df))
            ))
        elif self.embedding_mode == 'Glove':
            text_df = self.tokenize(raw_df,name)
            corpus_model = self.corpus_model[name]
            corpus_model.fit(text_df,window=10,ignore_missing=False)
            embedding.fit(corpus_model.matrix,epochs=10)
            embedding.add_dictionary(corpus_model.dictionary)
            embedded_text = sparse.csr_matrix(np.array(
                list(map(lambda x:get_words_glove_vec(embedding,x),text_df))
            ))
        elif self.embedding_mode == 'Bert':
            model = AutoModel.from_pretrained(self.embeddings[name],output_hidden_states=True)
            encoded_input = TrainDataset(self.tokenizers[name],raw_df[name])
            embedded_text = sparse.csr_matrix(model_infer(model,encoded_input,self.device))
            # embedded_text = sparse.csr_matrix(model(**encoded_input)['hidden_states'][0].detach().to('cpu').numpy().mean(1))
            # embedded_text = sparse.csr_matrix(model(**encoded_input)[0].detach().numpy().mean(1))
        else:
            raise NotImplementedError
        return embedded_text
    def fit_encoders(self,df,y=None):
        if self.task == 'embedding':
            res_dict={}
            for column in self.text_columns_def:
                print(f'Updating column: {column} embeddings output')
                res_dict.update({column:self.emb_text(df,column)})
            return res_dict
#                 df[f"{column}_meta_feature"] = self.emb_text(df[column],column)

        elif self.task == 'supervise' or self.task == 'unsupervise':
            def micro_regressor(embedding_array,y):
                regressor = Ridge(random_state=42, alpha=0.8)
                regressor.fit(embedding_array, y)
                return regressor
            def micro_cluster(embedding_array):
                cluster = KMeans(n_clusters=self.n_clusters)
                cluster.fit(embedding_array)
                return cluster
            for column in self.text_columns_def:
                print(f'Fitting column: {column} encoder')
                encoders = []
                folds = KFold(n_splits=5, shuffle=True, random_state=42)
                for fold_n, (train_index, valid_index) in enumerate(folds.split(df)):
                    if self.embedding_mode != 'Bert':
                        trn =  df[f'{column}_tokenized_ids'][train_index]
                        vld =  df[f'{column}_tokenized_ids'][valid_index]
                    else:
                        trn = TrainDataset(self.tokenizers[column],df[column][train_index])
                        vld = TrainDataset(self.tokenizers[column],df[column][valid_index])
                    if self.embedding_mode=='TFIDF':
                        trn = self.embeddings[column].transform(trn)
                        vld = self.embeddings[column].transform(vld)
                    elif self.embedding_mode=='FastText':
                        trn = sparse.csr_matrix(self.embeddings[column].wv[(trn.apply(tokens2sentence))])
                        vld = self.embeddings[column].wv[(vld.apply(tokens2sentence))]
                    elif self.embedding_mode=='Word2Vec':
                        trn = sparse.csr_matrix(np.array(list(map(lambda x:get_words_w2v_vec(self.embeddings[column],x),trn))))
                        vld = np.array(list(map(lambda x:get_words_w2v_vec(self.embeddings[column],x),vld)))
                    elif self.embedding_mode=='Glove':
                        trn = sparse.csr_matrix(np.array(list(map(lambda x:get_words_glove_vec(self.embeddings[column],x),trn))))
                        vld = np.array(list(map(lambda x:get_words_glove_vec(self.embeddings[column],x),vld)))
                    elif self.embedding_mode=='Bert':
                        model = AutoModel.from_pretrained(self.embeddings[column],output_hidden_states=True)
                        trn = sparse.csr_matrix(model_infer(model,trn,self.device))
                        vld = sparse.csr_matrix(model_infer(model,vld,self.device))
                        # trn = sparse.csr_matrix(model(**trn)['hidden_states'][0].detach().to('cpu').numpy().mean(1))
                        # vld = sparse.csr_matrix(model(**vld)['hidden_states'][0].detach().to('cpu').numpy().mean(1))
                    else:
                        raise NotImplementedError
                        
                    if self.task == 'supervise': 
                        y_trn = y.iloc[train_index]
                        encoders.append(micro_regressor(trn,y_trn))
                    else:
                        encoders.append(micro_cluster(trn))
                    
                    val = encoders[fold_n].predict(vld)
                    df.loc[valid_index, f"{column}_meta_feature"] = val
                self.encoders.update({column:encoders})
                if self.embedding_mode != 'Bert':
                    df = df.drop(columns=[f'{column}_tokenized_ids'])
            return df.drop(columns=self.text_columns_def)
    def transform(self,df):
        if self.task == 'embedding' :
            res_dict={}
            for column in self.text_columns_def:
                print(f'Updating column: {column} embeddings output')
                res_dict.update({column:self.emb_text(df,column)})
                # df[f'{column}_embedded'] = self.emb_text(df,column)
            return res_dict
        if self.task == 'supervise' or self.task == 'unsupervise':
            for column in self.text_columns_def:
                print(f'Transforming column: {column}')
                embedded_text = self.emb_text(df,column)
                meta_test = None
                if self.task == 'supervise':
                    for idx in range(5):
                        encoder = self.encoders[column][idx]
                        pred = (encoder.predict(embedded_text))/5
                        if idx==0:
                            meta_test = pred
                        else:
                            meta_test += pred
#                     df[f'{column}_transformed'] = meta_test
                else:
                    for idx in range(5):
                        encoder = self.encoders[column][idx]
                        pred = np.eye(self.n_clusters)[encoder.predict(embedded_text)]
                        if idx==0:
                            meta_test = pred
                        else:
                            meta_test += pred
                    meta_test = np.argmax(meta_test,axis=1)
#                     meta_test = pd.DataFrame(np.eye(self.n_clusters)[np.argmax(meta_test,axis=1)])
#                     for idx in range(self.n_clusters):
#                         df[f'{column}_transformed_class{idx}'] =  meta_test[idx]
                df[f'{column}_meta_feature'] = meta_test
            return df
        elif self.task == 'zero-shot-classification':
            for column in self.text_columns_def:
                pred_labels=[]
                classifier = self.pipeline
                candidate_labels = self.candidate_labels[column]
                for text in df[column]:
#                     classifier = self.pipline[]
                    text = text[:128]
                    results = classifier(text, candidate_labels)
                    pred_labels.append(results['labels'][np.argmax(results['scores'])])
                df[f'{column}_meta_feature'] = pred_labels
            return df
        else:
            raise NotImplementedError

In [ ]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv')
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
nlp = NLP_feature()
# nlp.tokenizers
nlp.do_mlm = True
nlp.emb_size=100
nlp.n_clusters=5
df = nlp.fit(df_train,['text'],True,'Bert','supervise',df_train['target'],None)

In [ ]:
df_train['text_meta_feature'] = df['text_meta_feature']

In [ ]:
print(nlp.tokenizers,'\n',nlp.embeddings,'\n',nlp.encoders)

In [ ]:
test = nlp.transform(df_test)

In [ ]:
df_train.to_csv('nlp_trn.csv',index=False)
test.to_csv('nlp_tst.csv',index=False)

# Lab

In [ ]:
model_name = 'prajjwal1/bert-tiny'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
text_df = df_train['comment_text']
text_df = text_df.apply(lambda x: x.replace('\n',''))
text  = '\n'.join(text_df.tolist())
with open('text.txt','w') as f:
    f.write(text)
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=128)
#                 valid_dataset = LineByLineTextDataset(
#                     tokenizer=self.tokenizers[name],
#                     file_path="text.txt", #mention train text file here
#                     block_size=128)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
training_args = TrainingArguments(
                    output_dir='output', #select model path for checkpoint
#                     overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy= 'no',
    save_strategy='no',
    report_to = "none")
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=None)
trainer.train()

In [ ]:
trainer.save_model('./output')

In [ ]:
model = AutoModel.from_pretrained(model_name,output_hidden_states=True)

In [ ]:
encoded_input = tokenizer(text_df.to_list(),max_length=256, return_tensors='pt',padding='max_length',truncation=True)
output = model(**encoded_input)

In [ ]:
encoded_input['input_ids'].shape

In [ ]:
sparse.csr_matrix(output['hidden_states'][0].detach().numpy().mean(1).shape)

## Load Datasets

In [ ]:
TRAIN_DATA_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
VALID_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv"
TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_valid = pd.read_csv(VALID_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)

In [ ]:
df_train=df_train.sample(n=10).reset_index(drop=True)

# Scoring training data

In [ ]:
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].mean(axis=1)

df_train['y'] = df_train['score']

# min_len = (df_train['y'] > 0).sum()  # len of toxic comments
# df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments
# df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df
# df_train_new

# Train the tokenizer

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('i love you'))

In [ ]:
def dummy_fun(doc):
    return doc

In [ ]:
len(tokenized_comments)

In [ ]:
labels = df_train_new['y']
comments = df_train_new['comment_text']
def id2token(tokenizer,ids):
    return tokenizer.convert_ids_to_tokens(ids)
    
tokenized_comments = [tokenizer.convert_ids_to_tokens(ids) for ids in tokenizer(comments.to_list())['input_ids']]

vectorizer = TfidfVectorizer(
    analyzer = 'word',
    tokenizer = dummy_fun,
    preprocessor = dummy_fun,
    token_pattern = None)

comments_tr = vectorizer.fit(tokenized_comments)
comments_tr

In [ ]:
# comments_tr.transform(['adwada','weqw']).toarray().tolist()

In [ ]:
# np.where((comments_tr.toarray()[0])!=0)
# vectorizer.get_feature_names()